In [3]:
# Initialize counters for the words "Israel" and "Iran"
israel_count = 0
iran_count = 0
netanyahu_count = 0
khamenei_count = 0
# Open the file in read mode
with open('/content/News Stories - Israel Iran (1).txt', 'r') as file:
    # Read each line in the file
    for line in file:
        # Split the line into words
        words = line.split()
        # Loop through each word
        for word in words:
            # Check if the word is "Israel" or "Iran" (case-insensitive)
            if word.lower() == "israel":
                israel_count += 1
            elif word.lower() == "iran":
                iran_count += 1
            elif word.lower() == "netanyahu":
              netanyahu_count += 2
            elif word.lower() == "khamenei":
              khamenei_count += 2

# Print the counts
print("Israel count:", israel_count)
print("Iran count:", iran_count)
print("Netanyahu count:", netanyahu_count)
print("Khamenei count:", khamenei_count)

Israel count: 500
Iran count: 453
Netanyahu count: 112
Khamenei count: 30


In [17]:
import pandas as pd
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

# Download the VADER lexicon (if not already downloaded)
nltk.download('vader_lexicon')

# Create a DataFrame from the sample data
df = pd.read_csv("/content/data.csv", encoding='ISO-8859-1')

# Filter the data to only include rows where the country is Israel or Iran
df = df[df['country'].isin(['Israel', 'Iran'])]

# Initialize the VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Function to calculate sentiment scores for each sentence
def get_sentiment_scores(text):
    scores = sid.polarity_scores(text)
    return scores

# Set the sample size and number of iterations
sample_size = 10
num_iterations = 50

# Initialize lists to store the sentiment scores for each iteration
israel_scores_list = []
iran_scores_list = []

# Loop over the number of iterations
for i in range(num_iterations):
    # Shuffle the data and split it into two subsets
    df = df.sample(frac=1).reset_index(drop=True)
    israel_df = df[df['country'] == 'Israel']
    iran_df = df[df['country'] == 'Iran']

    # Sample 50 rows from each subset
    israel_sampled_df = israel_df.sample(20, replace=True)
    iran_sampled_df = iran_df.sample(20, replace=True)

    # Concatenate the sampled subsets
    sampled_df = pd.concat([israel_sampled_df, iran_sampled_df], ignore_index=True)

    # Apply sentiment analysis to the text column
    sampled_df['sentiment_scores'] = sampled_df['text'].apply(get_sentiment_scores)

    # Calculate average sentiment scores for each subset
    israel_avg_scores = {
        'positive': sampled_df[sampled_df['country'] == 'Israel']['sentiment_scores'].apply(lambda x: x['pos']).mean(),
        'neutral': sampled_df[sampled_df['country'] == 'Israel']['sentiment_scores'].apply(lambda x: x['neu']).mean(),
        'negative': sampled_df[sampled_df['country'] == 'Israel']['sentiment_scores'].apply(lambda x: x['neg']).mean(),
        'compound': sampled_df[sampled_df['country'] == 'Israel']['sentiment_scores'].apply(lambda x: x['compound']).mean()
    }

    iran_avg_scores = {
        'positive': sampled_df[sampled_df['country'] == 'Iran']['sentiment_scores'].apply(lambda x: x['pos']).mean(),
        'neutral': sampled_df[sampled_df['country'] == 'Iran']['sentiment_scores'].apply(lambda x: x['neu']).mean(),
        'negative': sampled_df[sampled_df['country'] == 'Iran']['sentiment_scores'].apply(lambda x: x['neg']).mean(),
        'compound': sampled_df[sampled_df['country'] == 'Iran']['sentiment_scores'].apply(lambda x: x['compound']).mean()
    }

    # Append the sentiment scores to the lists
    israel_scores_list.append(israel_avg_scores)
    iran_scores_list.append(iran_avg_scores)

# Calculate the overall average sentiment scores
israel_avg_scores = {
    'positive': np.mean([x['positive'] for x in israel_scores_list]),
    'neutral': np.mean([x['neutral'] for x in israel_scores_list]),
    'negative': np.mean([x['negative'] for x in israel_scores_list]),
    'compound': np.mean([x['compound'] for x in israel_scores_list])
}

iran_avg_scores = {
    'positive': np.mean([x['positive'] for x in iran_scores_list]),
    'neutral': np.mean([x['neutral'] for x in iran_scores_list]),
    'negative': np.mean([x['negative'] for x in iran_scores_list]),
    'compound': np.mean([x['compound'] for x in iran_scores_list])
}

# Print the average sentiment scores
print("Israel Average Sentiment Scores:")
print(israel_avg_scores)
print("\nIran Average Sentiment Scores:")
print(iran_avg_scores)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Israel Average Sentiment Scores:
{'positive': 0.07553599999999999, 'neutral': 0.747396, 'negative': 0.17707899999999996, 'compound': -0.1878989}

Iran Average Sentiment Scores:
{'positive': 0.05185399999999998, 'neutral': 0.769006, 'negative': 0.179129, 'compound': -0.28728149999999997}
